<a href="https://colab.research.google.com/github/jpdicarvalho/Modelos-Estatisticos-Preditivos/blob/main/Regress%C3%A3o_Polinomial_(graus_2%2C_3%2C_4)_Previs%C3%A3o_para_o_ano_de_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

# Função para treinar e avaliar modelo de Regressão Polinomial com diferentes graus
def train_and_evaluate_polynomial_model(data, regions, degree):
    results = []

    for region in regions:
        region_data = np.array(data[region], dtype=float)  # Garantir tipo float

        # Remover valores nulos para treinamento
        valid_indices = ~np.isnan(region_data)
        years_valid = np.array(data["Year"])[valid_indices].reshape(-1, 1)
        values_valid = region_data[valid_indices]

        # Criar termos polinomiais
        poly = PolynomialFeatures(degree=degree)
        years_poly = poly.fit_transform(years_valid)

        # Dividir os dados em treino (80%) e teste (20%)
        X_train, X_test, y_train, y_test = train_test_split(years_poly, values_valid, test_size=0.2, random_state=42)

        # Regressão Polinomial
        model = LinearRegression()
        model.fit(X_train, y_train)

        # Previsão para os dados de teste
        y_pred = model.predict(X_test)

        # Métricas de erro
        mae = mean_absolute_error(y_test, y_pred)
        mse = mean_squared_error(y_test, y_pred)
        rmse = np.sqrt(mse)
        r2 = r2_score(y_test, y_pred)

        # Previsão para 2023
        future_year_poly = poly.transform([[2023]])
        future_prediction = model.predict(future_year_poly)

        # Coeficientes do modelo
        intercept = model.intercept_
        coefficients = model.coef_

        # Armazenar os resultados
        results.append({
            "Região": region,
            "Grau do Polinômio": degree,
            "Intercepto": intercept,
            "Coeficientes": coefficients.tolist(),
            "R² Treino": model.score(X_train, y_train),
            "R² Teste": r2,
            "MAE": mae,
            "MSE": mse,
            "RMSE": rmse,
            "Produção Estimada em 2023": future_prediction[0]
        })

        # Gráfico para a região
        plt.figure(figsize=(6, 4))
        plt.scatter(years_valid, values_valid, color="blue", label="Dados reais")
        plt.plot(years_valid, model.predict(years_poly), color="red", label=f"Regressão Polinomial (Grau {degree})")
        plt.scatter(X_test[:, 1], y_test, color="orange", label="Dados de teste", zorder=5)
        plt.scatter(2023, future_prediction, color="green", label="Previsão 2023", zorder=6)
        plt.title(f"Regressão Polinomial (Grau {degree}) - {region}")
        plt.xlabel("Ano")
        plt.ylabel("Produção")
        plt.legend()
        plt.grid()
        plt.show()

    return pd.DataFrame(results)

# Testar diferentes graus de regressão polinomial
degrees = [2, 3, 4]  # Grau 2, 3 e 4
polynomial_results = {}

# Se estiver rodando no Jupyter Notebook ou Colab, use display() para melhor visualização
from IPython.display import display

for d in degrees:
    polynomial_results[d] = train_and_evaluate_polynomial_model(data, regions, degree=d)

# Exibir tabelas para diferentes graus de polinômios no ambiente correto
for d, df in polynomial_results.items():
    display(df)  # Substitui ace_tools para visualização no Colab/Jupyter

# Exibir tabela final com previsões da regressão polinomial quadrática
print(polynomial_results[2])  # Exibir previsões de grau 2
display(polynomial_results[2])  # Melhor visualização no Jupyter Notebook ou Colab